## **Set directories**

In [2]:
import os

tools_path = "./tools"
assert os.path.exists(tools_path), f"tools path does not exists: {tools_path}"

DATASETS = "/home/qjh/anomaly_data/datasets"
dtd_path = DATASETS + "/dtd"
mvtec_path = DATASETS + "/mvtec"
assert os.path.exists(dtd_path), f"DTD path does not exists: {dtd_path}"
assert os.path.exists(mvtec_path), f"MVTEC path does not exists: {mvtec_path}"

OUTPUT_PATH = "/home/qjh/anomaly_data/result"
assert os.path.exists(OUTPUT_PATH), f"DTD path does not exists: {OUTPUT_PATH}"


print("Directories set.")

Directories set.


## **Install the library required**

In [ ]:
%pip install -r {WORKDIR}/requirements.txt

## **Generate foreground and global retrieval result**

In [ ]:
!python {tools_path}/generate_foreground.py \
    -lp {OUTPUT_PATH}/foreground \
    --data_dir {DATASETS} \
    --vis

!python {tools_path}/generate_retrieval.py \
    -lp {OUTPUT_PATH}/retrieval \
    --data_dir {DATASETS} \
    --vis \
    -k 5


## **Train**

### ***Generate synthetic data***

In [ ]:
!python {tools_path}/generate_synthetic_data.py \
    -fd {OUTPUT_PATH}/foreground \
    -lp {OUTPUT_PATH}/synthetic_mvtec

### ***Run***

*Please set the OUTPUT_PATH which is same as before*

In [14]:
%%bash

# objects=('bottle' 'cable' 'capsule' 'hazelnut' 'metal_nut' 'pill' 'screw' 'toothbrush' 'transistor' 'zipper')
# textures=('carpet' 'grid' 'leather' 'tile' 'wood')
objects=('bottle')
OUTPUT_PATH="/home/qjh/anomaly_data/result"
DATASETS="/home/qjh/anomaly_data/datasets"

# [10] bottle test result: pro: 0.9593 ap: 0.8097 pixel-auc: 0.9819 image-auc: 1.0000
# [20] bottle test result: pro: 0.9619 ap: 0.8193 pixel-auc: 0.9836 image-auc: 1.0000
# [50] bottle test result: pro: 0.9701 ap: 0.8754 pixel-auc: 0.9872 image-auc: 1.0000

# python train.py -lp log/mvtec_train --sub-categories "${textures[@]}" \
# -fd ./log/foreground_mvtec_DenseNet_features.denseblock1_320/ \
# --steps 500 \
# -tps 50 \
# --data-dir log/synthetic_mvtec_640_12000_True_jpg

python train.py \
    -lp ${OUTPUT_PATH}/mvtec_train \
    --sub-categories "${objects[@]}" \
    -fd ${OUTPUT_PATH}/foreground/ \
    -rd ${OUTPUT_PATH}/retrieval \
    --steps 3 \
    -tps 1 \
    --data-dir ${OUTPUT_PATH}/synthetic_mvtec \
    --root_dir ${DATASETS}

2025-03-12 04:52:01.969 | INFO     | __main__:main:101 - run argv: train.py -lp /home/qjh/anomaly_data/result/mvtec_train --sub-categories bottle -fd /home/qjh/anomaly_data/result/foreground/ -rd /home/qjh/anomaly_data/result/retrieval --steps 3 -tps 1 --data-dir /home/qjh/anomaly_data/result/synthetic_mvtec --root_dir /home/qjh/anomaly_data/datasets
2025-03-12 04:52:01.969 | INFO     | __main__:main:102 - args: 
{'T': 512,
 'batch_size': 32,
 'data_dir': '/home/qjh/anomaly_data/result/synthetic_mvtec',
 'dataset_name': 'mvtec',
 'foreground_dir': '/home/qjh/anomaly_data/result/foreground/',
 'k_nearest': 10,
 'learning_rate': 0.001,
 'log_path': '/home/qjh/anomaly_data/result/mvtec_train',
 'n_anomaly': 0,
 'num_workers': 8,
 'pretrained_model': 'DenseNet',
 'region_sizes': [3, 1],
 'resize': 320,
 'retrieval_dir': '/home/qjh/anomaly_data/result/retrieval',
 'root_dir': '/home/qjh/anomaly_data/datasets',
 'scales': [4, 8],
 'steps': 3,
 'sub_categories': ['bottle'],
 'test_per_steps':

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/densenet201.tv_in1k)
INFO:timm.models._hub:[timm/densenet201.tv_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
extract train local features: 100%|█████████▉| 208/209 [00:06<00:00, 35.23it/s]
                                                                               
predict test data: 100%|██████████| 83/83 [00:09<00:00,  9.41it/s]


2025-03-12 04:52:37.852 | INFO     | __main__:main:169 - [1] bottle test result: pro: 0.9622 ap: 0.8209 pixel-auc: 0.9827 image-auc: 1.0000 image-f1: 1.0000 f1-threshold: 1.0000


extract train local features:  99%|█████████▉| 207/209 [00:05<00:00, 37.01it/s]
                                                                               
predict test data:  99%|█████████▉| 82/83 [00:08<00:00, 10.41it/s]


2025-03-12 04:52:56.698 | INFO     | __main__:main:169 - [2] bottle test result: pro: 0.9629 ap: 0.8198 pixel-auc: 0.9832 image-auc: 1.0000 image-f1: 1.0000 f1-threshold: 1.0000


extract train local features: 100%|██████████| 209/209 [00:05<00:00, 36.26it/s]
                                                                               
predict test data: 100%|██████████| 83/83 [00:08<00:00, 10.56it/s]


2025-03-12 04:53:15.669 | INFO     | __main__:main:169 - [3] bottle test result: pro: 0.9620 ap: 0.8175 pixel-auc: 0.9830 image-auc: 1.0000 image-f1: 1.0000 f1-threshold: 1.0000


## **Test**

In [ ]:
!python test.py \
    -lp result/mvtec_test \
    -fd result/foreground/ \
    -rd result/retrieval \
    --checkpoints weights/{category}.pth

## **Open the mlflow ui**

In [ ]:
import mlflow

!mlflow ui --backend-store-uri file:'./mlruns' 